In [1]:
from copy import deepcopy as copy
from numpy import linalg
import numpy as np
from math import sqrt
from pandas import DataFrame as df
from math import log

In [2]:
def apply(a, v):
    res = copy(v)
    for i in range(len(a)):
        res[i] = 0
        for j in range(len(a)):
            res[i] += a[i][j] * v[j]
    return res

def vsum(v1, v2):
    res = copy(v1)
    for i in range(len(v1)):
        res[i] += v2[i]
    return res

def smul(x, v1):
    return [x * y for y in v1]

# 2.  Итерационные методы решения СЛАУ

In [3]:
A = [[9.016024, 1.082197, -2.783575],
     [1.082197, 6.846595, 0.647647],
     [-2.783575, 0.647647, 5.432541]]

B = [-1.340873, 4.179164, 5.478007]

n = len(A)

eps = 0.0001

### Точное решение системы, полученное методом Гаусса

In [4]:
x_gauss = [0.10000015652854335, 0.49999993775826096, 1.0000000876237503]

### построим H и g

In [5]:
H = [[0] * n for i in range(n)]
for i in range(n):
    for j in range(n):
        if i != j:
            H[i][j] = -A[i][j] / A[i][i]
g = [0] * n
for i in range(n):
    g[i] = B[i] / A[i][i]

In [6]:
H

[[0, -0.12003040364577558, 0.30873642306187293],
 [-0.15806353377116655, 0, -0.09459402812638983],
 [0.5123891379742923, -0.11921621944500742, 0]]

In [7]:
g

[-0.14872109923398608, 0.6104003522919057, 1.0083691959250745]

### Вычислим спектральный радиус

In [8]:
spectral_radix = max([abs(x) for x in linalg.eig(H)[0]])
spectral_radix

0.46218782245536616

### Функция для вычисления априорной оценки необходимого количества итераций

In [9]:
# || v ||_inf 
def vnorm(v):
    return max(v)

# || A ||_inf
def norm(A):
    return max([sum(row) for row in A])

def its_estimated(H, g, x0, eps):
    return log(eps / (vnorm(x0) + vnorm(g) / (1 - norm(H)))) / log(norm(H))

Вычислим норму H

In [10]:
norm(H)

0.3931729185292849

Норма < 1, то есть удовлетворяет достаточному условию сходимости.
Спектральный радиус также < 1, значит H соответствует необходимому и достаточному условию сходимости.

### Уточнение по Люстернику

In [11]:
def lusternik(x_prev, x_last):
    return vsum(x_prev, smul(1./(1 - spectral_radix), vsum(x_it, smul(-1, x_prev))))

## Метод простой итерации

Априорная оценка количества итераций:

In [12]:
its_estimated(H, g, g, eps)

10.918462396295723

In [13]:
x_it = copy(g)
x_prev = copy(x_it)
cnt_it = 0
while max([abs(x_it[i] - x_gauss[i]) for i in range(n)]) > eps:
    cnt_it += 1
    x_prev = copy(x_it)
    x_it = vsum(g, apply(H, x_it))

решение:

In [14]:
x_it

[0.09992898343723441, 0.5000336332891339, 0.9999427428043615]

количество итераций:

In [15]:
cnt_it

10

уточнение по Люстернику:

In [16]:
x_it_l = lusternik(x_prev, x_it)
x_it_l

[0.09994822736073213, 0.49999480986808165, 1.000065224005445]

## Метод Зейделя

Априорная оценка количества итераций:

In [17]:
HL = copy(H)
HR = copy(H)
for i in range(n):
    for j in range(n):
        if j < i:
            HL[i][j] = H[i][j]
            HR[i][j] = 0
        else:
            HL[i][j] = 0
            HR[i][j] = H[i][j]
E = copy(H)
for i in range(n):
    for j in range(n):
        if i == j:
            E[i][j] = 1
        else:
            E[i][j] = 0
            
HL = np.array(HL)
HR = np.array(HR)
E = np.array(E)

Hseid = linalg.inv(E - HL).dot(HR)
gseid = linalg.inv(E - HL).dot(g)

its_estimated(Hseid, gseid, g, eps)

5.957840592227704

In [18]:
x_z = copy(g)
x_prev = copy(x_z)
cnt_z = 0
while max([abs(x_z[i] - x_gauss[i]) for i in range(n)]) > eps:
    cnt_z += 1
    x_prev = copy(x_z)
    x_z = [0] * n
    for i in range(n):
        for j in range(i):
            x_z[i] += H[i][j] * x_z[j]
        for j in range(i, n):
            x_z[i] += H[i][j] * x_prev[j]
        x_z[i] += g[i]

решение:

In [19]:
x_z

[0.0999084898582892, 0.5000364914092834, 0.9999487608295159]

количество итераций:

In [20]:
cnt_z

4

уточнение по Люстернику:

In [21]:
x_z_l = lusternik(x_prev, x_z)
x_z_l

[0.10022629797427937, 0.49992188432100176, 1.0000939170743568]

## Метод верхней релаксации

In [22]:
q = 2./(1 + sqrt(1 - spectral_radix**2))

x_sor = copy(g)
x_prev = copy(x_sor)
cnt_sor = 0
while max([abs(x_sor[i] - x_gauss[i]) for i in range(n)]) > eps:
    cnt_sor += 1
    x_prev = copy(x_sor)
    for i in range(n):
        x_sor[i] += q * g[i]
        x_sor[i] -= q * x_prev[i]
        for j in range(i):
            x_sor[i] += q * H[i][j] * x_sor[j]
        for j in range(i, n):
            x_sor[i] += q * H[i][j] * x_prev[j]

решение:

In [23]:
x_sor

[0.10001943593120183, 0.4999976287341581, 1.0000079353642581]

количество итераций:

In [24]:
cnt_sor

4

## Сравнение результатов

In [25]:
def max_error(v1):
    return max([abs(v1[i] - x_gauss[i]) for i in range(len(v1))])

def mean_error(v1):
    return sum([abs(v1[i] - x_gauss[i]) for i in range(len(v1))]) / len(v1)

table = df(data={
    'Метод': ['Простой итерации', 'Простой итерации с уточнением', 'Зейделя', 'Зейделя с уточнением', 'Верхней релаксации'],
    'Максимальная ошибка': [max_error(x) for x in [x_it, x_it_l, x_z, x_z_l, x_sor]],
    'Средняя ошибка': [mean_error(x) for x in [x_it, x_it_l, x_z, x_z_l, x_sor]],
    'Число итераций': [cnt_it, cnt_it, cnt_z, cnt_z, cnt_sor]
})
table.style.hide_index()

Метод,Максимальная ошибка,Средняя ошибка,Число итераций
Простой итерации,0.000071,0.000054,10
Простой итерации с уточнением,0.000065,0.000041,10
Зейделя,0.000092,0.000060,4
Зейделя с уточнением,0.000226,0.000133,4
Верхней релаксации,0.000019,0.000010,4
